In [ ]:
%matplotlib inline
from __future__ import print_function, absolute_import

### 1) No-op

Step 1) Vanilla dircol traj. optimization.

In [5]:
from dircol import make_real_dircol_mp

dircol, tree = make_real_dircol_mp(expmt="cartpole", seed=1776)

R = 10  # Cost on input "effort".
u = dircol.input()
dircol.AddRunningCost(R*u[0]**2)

Step 2) Simple supervised learning on (x, u) pairs, with no constraints on policy.

### 2) GPS

Step 1) dircol traj. optimization using Augmented Lagrangian

In [ ]:
def cost():
    pass
def Pi():
    pass
def lagrangian_t(x, u, theta, lam, rho=1): # TODO: figure out rho...
    global cost
    global Pi
    a = cost(x, u)                # Additive cost
    b = lam[t]*(Pi(x[t])-u[t])    # Model L1 penalty?
    c = rho[t]*(Pi(x[t])-u[t])**2 # Model L2 penalty?
    return a+b+c

In [8]:
from dircol import make_real_dircol_mp

dircol, tree = make_real_dircol_mp(expmt="cartpole", seed=1776)

R = 10  # Cost on input "effort".
u = dircol.input()
dircol.AddRunningCost(R*u[0]**2)

from NNSystemHelper import FC, MLP, NNInferenceHelper
# net = FC().double()
net = MLP().double()
def autodiffable_function_simple(x):
    return (x[0]-2.)*(x[1]-2.)
def autodiffable_function_nn(drake_in):
    global net
    in_list = list(drake_in)
    out_list = NNInferenceHelper(net, in_list)
    return out_list[0]

# Add on an autodiffable function that can query a torch model!
def autodiffable_function_nn_real(state_concat_inp):
    # Unpack variables
    x_sz = len(prog.state(0))
    u_sz = len(prog.input(0))
    assert len(state_concat_inp) == x_sz + u_sz
    x = state_concat_inp[:x_sz]
    u = state_concat_inp[-u_sz:]
    
    # Should we use an NN? or not.
    ad_fn = autodiffable_function_simple
#     ad_fn = autodiffable_function_nn
    
    lam, rho = 1, 1 # For now, set the dual variables to constants
    cost = lam * (ad_fn(x) - u) \
        + rho * (ad_fn(x) - u).dot(ad_fn(x) - u)
    return cost[0]

# prog.AddCost(autodiffable_function_nn_real, np.hstack((prog.state(0), prog.input(0)))) # For debugging
# Looking to add cost of form 𝜆_𝑡(𝜋_𝜃(𝑥_𝑡) − 𝑢_𝑡) + 𝜌_𝑡(𝜋_𝜃(𝑥_𝑡) − 𝑢_𝑡)^2
for t in range(kNumTimeSteps):
    prog.AddCost(autodiffable_function_nn_real, np.hstack((prog.state(t), prog.input(t))))
    pass

ImportError: No module named NNSystemHelper

Step 2) Supervised learning on (x, u) pairs, using Augmented Lagrangian

### 3) Interactive Control

Step 1) dircol traj. optimization with L2 augmentation and L2 trajectory distance penalty.

In [7]:
from dircol import make_real_dircol_mp

dircol, tree = make_real_dircol_mp(expmt="cartpole", seed=1776)

R = 10  # Cost on input "effort".
u = dircol.input()
dircol.AddRunningCost(R*u[0]**2)

Step 2) Simple supervised learning on (x, u) pairs, with L2 augmentation and L2 policy distance penalty.